In [ ]:
%pip install LangChain-community
%pip install chromadb
!ollama pull llama3
!ollama pull nomic-embed-text

In [ ]:
from langchain_community.llms import Ollama

ollama = Ollama(
    base_url = "http://localhost:11434",
    model="llama3"
)

In [ ]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.gutenberg.org/files/1727/1727-h/1727-h.htm")
data = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [ ]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=oembed)

In [ ]:
question = "Who is Neleus and who is in Neleus's family?"
docs = vectorstore.similarity_search(question)
len(docs)

In [ ]:
from langchain.chains import RetrievalQA
qachain = RetrievalQA.from_chain_type(ollama, retriever=vectorstore.as_retriever())
res = qachain.invoke({"query": question})
print(res['result'])